In [41]:
import numpy as np
import urllib2
import csv
import pandas as pd
from subprocess import call
from scipy.interpolate import UnivariateSpline
import matplotlib.pyplot as plt


mean = 0.0
std = 0.0

initialYear = 2000
endyear = 2004
years = 4
city = "calgary"
code = "113"
# maxT = 0.0


def makeCSV(year):
    csvfile = urllib2.urlopen('http://climate.weather.gc.ca/climateData/bulkdata_e.html?format=csv&stationID='+code+'&Year='+str(year)+'&Month=1&Day=1&timeframe=2&submit=Download+Data')
    csvResponse = csvfile.read()

    target = open("temp.csv", 'w')

    target.write(csvResponse)

    mainRows = []
    with open('temp.csv') as csvfile:
        csvfile = csv.reader(csvfile)
        for row_no,row in enumerate(csvfile):
            if row_no > 24:
                mainRows.append(row)

    call(["rm","temp.csv"])

    filename = city+str(year)+'.csv'
    with open(filename, 'w') as fp:
        a = csv.writer(fp, delimiter=',')
        a.writerows(mainRows)
            
def getFiveYearsMean(year):
    filename = city+year+'.csv'
    data = pd.read_csv(filename)
    global mean
    global std

    meanTempDay = data[data.columns[9]]
    mean =mean + round(meanTempDay.mean(),2)/years
    std = std + round(meanTempDay.std(),2)/years
    # maxT = meanTempDay.max()

    l = []
    for i in xrange(24):
        l.append( (meanTempDay[15*i:15*i+15].mean() ) )

    return pd.DataFrame(l)/years


def getBSplineCurve(fifteenDaysMean):
    x = np.linspace(0,23,24)
    y = np.array(fifteenDaysMean)
    plt.plot(x, y,'ro', ms= 5)
    plt.xlabel('15 days mean')
    plt.ylabel('temperature')
    plt.title(city+str(initialYear)+'-'+str(endyear))

    spl = UnivariateSpline(x, y)
    xs = np.linspace(0, 23, 1000)
    plt.plot(xs, spl(xs), 'g', lw=3)

#     plt.text(1,25,'mu = '+ str(mean)+' sigma = '+ str(std), horizontalalignment='left',verticalalignment='center')
    plt.show()
    

for curYear in range(initialYear,endyear+1):
    makeCSV(curYear)


fifteenDaysMean = None
for curYear in range(initialYear,endyear+1):
    if fifteenDaysMean is None:
        fifteenDaysMean = getFiveYearsMean(str(curYear))
    else:
        fifteenDaysMean = fifteenDaysMean + getFiveYearsMean(str(curYear))
    

In [43]:
fifteenDaysMean

,0
0,7.916667
1,7.233333
2,7.873333
3,8.385000
4,9.121667
5,10.213333
6,12.018333
7,13.263333
8,13.966667
9,16.280000


In [44]:
fifteenDaysMean.fillna(fifteenDaysMean.mean())

,0
0,7.916667
1,7.233333
2,7.873333
3,8.385000
4,9.121667
5,10.213333
6,12.018333
7,13.263333
8,13.966667
9,16.280000


In [56]:
fifteenDaysMean.loc[0] = 5

In [57]:
fifteenDaysMean

,0
0,5.000000
1,7.233333
2,7.873333
3,8.385000
4,9.121667
5,10.213333
6,12.018333
7,13.263333
8,13.966667
9,16.280000
